In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Goal
The goal of this notebook is essentially to recommend a driver's lifetime value (LTV).


In [0]:
import os

# change to path
PATH='/content/drive/My Drive/Colab Notebooks/lyft'
os.chdir(PATH)

DATA_DIR = os.path.join(PATH, 'data')

In [0]:
import pandas as pd
import numpy as np

from datetime import datetime

import warnings
warnings.simplefilter(action='ignore')

## Read in

In [0]:
ride_timestamps = pd.read_csv(os.path.join(DATA_DIR, 'ride_timestamps.csv'))
ride_ids = pd.read_csv(os.path.join(DATA_DIR, 'ride_ids.csv'))
driver_ids = pd.read_csv(os.path.join(DATA_DIR, 'driver_ids.csv'))

In [5]:
ride_timestamps.head()

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


In [6]:
ride_ids.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


In [7]:
driver_ids.head()

,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


## Hypotheses
1. The longer the driver is onboard (until the last day the `driver_id` appears in the `ride_timestamps`), the higher his or her LTV.
2. Driving more frequently (or full-time or continuously picking up, will be the `drop_requested_lag` feature later) corresponds to higher LTV.
3. Longer distance corresponds to higher LTV.
4. Longer duration corresponds to higher LTV. (This alludes to the more the driver make the higher its LTV.)
5. Higher Prime Time (high demand hours) corresponds to higher LTV. 
6. Short `requested_at` and `accepted_at` window (`delta_requested_at` feature later) corresponds to higher LTV.




## EDA

### `drivers_ids`

In [8]:
driver_ids['driver_id'].nunique() # len(driver_ids)

937

In [9]:
driver_ids['driver_onboard_date'].min(), driver_ids['driver_onboard_date'].max()

('2016-03-28 00:00:00', '2016-05-15 00:00:00')

The file `driver_ids` consists of drivers who signed up as a driver with Lyft between 2016/03/28 to 2016/05/15.

### `ride_ids`

In [10]:
ride_id_g = ride_ids.groupby('driver_id')['ride_id'].count()

ride_id_g = ride_id_g.reset_index()

ride_id_g.sort_values(by='ride_id', ascending=False).head()

,driver_id,ride_id
354,5ccc0e6dc9c7475caf785cdce7b8eb7a,919
308,4eb382d1f7d50fae1294964263d1ce82,831
509,844e9be5a30d8d9c1f8e9ddb086ff717,821
401,689bdf87fb2de49f98bf4946cfaa5068,794
217,3788dc9e91f1548816ce8b5af07ddadc,783


In [11]:
len(ride_id_g)

937

Check whether do the rides start after all the drivers have registered.

In [0]:
ride_timestamps['timestamp'] = pd.to_datetime(ride_timestamps['timestamp'])

In [13]:
ride_timestamps['timestamp'].min()

Timestamp('2016-03-28 05:48:18')

Apparently, according to `timestamp` in `ride_timestamps`, some drivers who have registered earlier started driving earlier.

In [14]:
ride_timestamps['timestamp'].max()

Timestamp('2016-06-27 00:50:50')

The dataset for rides ended on 2016/06/27.

In [0]:
def seconds_lag(t):
    delta_to_t0 = t - t.iloc[0]
    return delta_to_t0

In [16]:
%%time
ride_timestamps['delta_requested_at'] = ride_timestamps.groupby('ride_id')['timestamp'].apply(seconds_lag)

ride_timestamps['delta_requested_at'] = ride_timestamps['delta_requested_at'].apply(lambda o: o.total_seconds())

CPU times: user 2min 28s, sys: 3.97 s, total: 2min 32s
Wall time: 2min 27s


In [17]:
ride_timestamps.head()

,ride_id,event,timestamp,delta_requested_at
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,0.0
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,32.0
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,312.0
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,314.0
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,1426.0


In [18]:
accepted_filter = ride_timestamps[ride_timestamps['event'] == 'accepted_at']

driver_accepted_filter = pd.merge(accepted_filter, ride_ids,
                                  on='ride_id')

window_g = driver_accepted_filter.groupby('driver_id')['delta_requested_at'].mean()

window_g = window_g.reset_index()

window_g.head()

,driver_id,delta_requested_at
0,002be0ffdc997bd5c50703158b7c2491,8.407942
1,007f0389f9c7b03ef97098422f902e62,11.838710
2,011e5c5dfc5c2c92501b8b24d47509bc,7.294118
3,0152a2f305e71d26cc964f8d4411add9,14.198953
4,01674381af7edd264113d4e6ed55ecda,12.528000


In [19]:
len(window_g)

844

In [20]:
ride_timestamps['ride_id'].nunique(), ride_ids['ride_id'].nunique()

(194081, 193502)

Apparently, some rides are not recorded in `ride_ids`, meaning some rides have `timestamp`s but not information of drivers.

## `ride_ids`

In [0]:
ride_ids['ride_distance'] = ride_ids['ride_distance'] / 1609.34

ride_ids['ride_duration'] = ride_ids['ride_duration'] / 60

Calculate fare.

In [22]:
ride_ids['price'] = 2 + \
                    ride_ids['ride_distance'] * 1.15 + \
                    ride_ids['ride_duration'] * 0.22 + \
                    1.75 

ride_ids['price'].describe()

def limit_bound(price):
    return min(max(5, price), 400)

ride_ids['price'] = ride_ids['price'].apply(limit_bound)

ride_ids['price'].describe()

count    193502.000000
mean         11.874203
std           8.061357
min           5.000000
25%           7.420821
50%           9.419351
75%          12.852304
max         400.000000
Name: price, dtype: float64

In [23]:
price_g = ride_ids.groupby('driver_id')['price'].sum()

price_g = price_g.reset_index()

price_g.sort_values(by='price', ascending=False).head()

,driver_id,price
354,5ccc0e6dc9c7475caf785cdce7b8eb7a,10732.268810
217,3788dc9e91f1548816ce8b5af07ddadc,10231.462474
509,844e9be5a30d8d9c1f8e9ddb086ff717,9357.989628
308,4eb382d1f7d50fae1294964263d1ce82,9275.832250
722,c07499b5a6f1090f2fb263ec6ac0660c,8955.817872


The higher a driver's `ride_prime_time`, the more it is valuable to Lyft since they help to meet passengers' demand.

In [24]:
# make the variable NaN to calculate mean
ride_ids['ride_prime_time'].replace(0, np.nan, inplace=True)

ride_prime_time_g = ride_ids.groupby('driver_id')['ride_prime_time'].mean()

ride_prime_time_g = ride_prime_time_g.reset_index()

ride_prime_time_g.sort_values(by='ride_prime_time', ascending=False).head()

,driver_id,ride_prime_time
807,dae249fc394c9bdf02f7d8bb1ff55733,146.875000
676,b2d3f2fb171a12cac427107690c10089,105.000000
420,6cb35e276085548f3f095a85aa63af7b,98.076923
443,7419cd5c573ff9994c0f8ff5d92b4408,91.071429
655,acd7dc6118befb6724aa3752d1cdbea1,90.277778


In [25]:
len(ride_prime_time_g)

937

Merge both `driver_ids` and `driver_timestamps`.

In [26]:
ids_timestamps = pd.merge(ride_ids, ride_timestamps,
                          left_on='ride_id',
                          right_on='ride_id')

ids_timestamps = ids_timestamps.sort_values(by=['driver_id', 'timestamp'])

ids_timestamps.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,price,event,timestamp,delta_requested_at
1235,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,requested_at,2016-03-29 18:46:50,0.0
1236,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,accepted_at,2016-03-29 18:47:01,11.0
1237,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,arrived_at,2016-03-29 18:50:00,190.0
1238,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,picked_up_at,2016-03-29 18:50:01,191.0
1239,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,dropped_off_at,2016-03-29 18:55:47,537.0


In [27]:
ids_timestamps['driver_id'].nunique()

844

In the original `driver_ids`, we have 937 unique `driver_ids` but after merging the `ride_ids` and `ride_timestamps`, there are only 844 unique `driver_ids`. We can conclude that some `ride_timestamp`s are absent for 93 drivers.

Create a feature `drop_requested_lag` to measure how fast do drivers pick up the next ride after they drop off passengers. The feature `drop_requested_lag` only considers same day pickups. So there exists a condition that a driver only picks up a passenger a day and this might lead to low `drop_requested_lag` value.

In [28]:
ids_timestamps['timestamp_shift'] = ids_timestamps.groupby('driver_id')['timestamp'].apply(lambda o: o.shift())

ids_timestamps['delta_timestamp'] = ids_timestamps['timestamp'] - ids_timestamps['timestamp_shift']

ids_timestamps.head(6)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,price,event,timestamp,delta_requested_at,timestamp_shift,delta_timestamp
1235,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,requested_at,2016-03-29 18:46:50,0.0,NaT,NaT
1236,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,accepted_at,2016-03-29 18:47:01,11.0,2016-03-29 18:46:50,00:00:11
1237,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,arrived_at,2016-03-29 18:50:00,190.0,2016-03-29 18:47:01,00:02:59
1238,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,picked_up_at,2016-03-29 18:50:01,191.0,2016-03-29 18:50:00,00:00:01
1239,002be0ffdc997bd5c50703158b7c2491,e228d93dc13a2a9d83661321992cd48e,1.541004,5.766667,NaN,6.790822,dropped_off_at,2016-03-29 18:55:47,537.0,2016-03-29 18:50:01,00:05:46
5,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,2.089055,13.483333,NaN,9.118747,requested_at,2016-03-29 19:00:49,0.0,2016-03-29 18:55:47,00:05:02


In [0]:
ids_timestamps.loc[(ids_timestamps['event'] == 'requested_at') &
                   (~ids_timestamps['delta_timestamp'].isna()), 'drop_requested_lag'] = 1

ids_timestamps['drop_requested_lag'].fillna(0, inplace=True)

ids_timestamps['drop_requested_lag'] = ids_timestamps['drop_requested_lag'] * ids_timestamps['delta_timestamp']

ids_timestamps['drop_requested_lag'] = ids_timestamps['drop_requested_lag'].apply(lambda o: o.total_seconds())

ids_timestamps.loc[ids_timestamps['delta_timestamp'].dt.days > 0, 'drop_requested_lag'] = np.nan

In [0]:
# ids_timestamps.head(31)

In [31]:
filtered_requested_at = ids_timestamps[ids_timestamps['event'] == 'requested_at']

drop_requested_lag_g = filtered_requested_at.groupby('driver_id')['drop_requested_lag'].mean()

drop_requested_lag_g = drop_requested_lag_g.reset_index()

drop_requested_lag_g.sort_values(by='drop_requested_lag').head()

,driver_id,drop_requested_lag
441,7ff85c5c0e9324e28d1e0d0589c364bd,181.000000
526,9a54684a69721c1075c2af5fc077665b,754.351351
495,905f8007cd46415eba8b9dce088b4395,936.774194
226,42256e33936dfa69088f540a720edc97,942.870968
313,5cf93f7d1d3a8f0cf395c84053c31b1b,976.818182


Merge `ids_timestamps` with `driver_id`.

In [0]:
df = pd.merge(ids_timestamps, driver_ids,
              left_on='driver_id', right_on='driver_id')

Create the feature `driving_period` to account for the duration between the last trip a driver made and the day the driver was onboard.

In [33]:
df['driving_period'] = (df.groupby('driver_id')['timestamp'].tail(1) - pd.to_datetime(df['driver_onboard_date'])).dt.days

onboard_period = df[~df['driving_period'].isna()]

onboard_period = onboard_period[['driver_id', 'driving_period']].copy()

onboard_period.sort_values('driving_period').head()

,driver_id,driving_period
714594,bd057e02f75c92917389d90bb215fe91,1.0
404069,68b546b2f4102641d6774c1ce4f57457,1.0
428659,6eb2e0f41fc9f1fbb70e7751035fdf87,1.0
608379,a2334fdb829cf96fbae920df0cce1587,2.0
839434,e4f3a9d5cf57b9b518136afd9757f76d,3.0


In [0]:
ltv = ride_id_g

to_merge = (drop_requested_lag_g, ride_prime_time_g, price_g, onboard_period, window_g)

for ser in to_merge:
    ltv = pd.merge(ltv, ser, on='driver_id', how='outer')

In [35]:
len(ltv)

937

In [36]:
ltv.isna().sum()

driver_id               0
ride_id                 0
drop_requested_lag     94
ride_prime_time         3
price                   0
driving_period        101
delta_requested_at     93
dtype: int64

To impute a feature's `NaN`s, if the higher the value of the feature the better, we impute with 0, otherwise we impute with 999,999. Note that having a `driving_period` of 0 indicates the possibility that a driver has signed up but never started to drive.

In [0]:
ltv['ride_prime_time'].fillna(0, inplace=True) # high better

ltv['driving_period'].fillna(0, inplace=True) # high better

ltv['drop_requested_lag'].fillna(999_999, inplace=True) # low better

ltv['delta_requested_at'].fillna(999_999, inplace=True) # low better

In [38]:
ltv.head()

,driver_id,ride_id,drop_requested_lag,ride_prime_time,price,driving_period,delta_requested_at
0,002be0ffdc997bd5c50703158b7c2491,277,6594.516129,48.863636,3094.039917,86.0,8.407942
1,007f0389f9c7b03ef97098422f902e62,31,4615.550000,52.083333,275.486554,85.0,11.838710
2,011e5c5dfc5c2c92501b8b24d47509bc,34,7185.041667,42.187500,427.273276,68.0,7.294118
3,0152a2f305e71d26cc964f8d4411add9,191,10028.796610,42.708333,2409.446265,64.0,14.198953
4,01674381af7edd264113d4e6ed55ecda,375,6633.404372,47.474747,4949.088065,56.0,12.528000


In [39]:
ltv.describe()

,ride_id,drop_requested_lag,ride_prime_time,price,driving_period,delta_requested_at
count,937.000000,937.000000,937.000000,937.000000,937.000000,937.000000
mean,206.512273,107718.730961,48.853483,2452.168669,49.184632,99263.550738
std,173.254063,298142.263520,10.236498,2049.864586,26.717681,299157.143154
min,3.000000,181.000000,0.000000,25.691325,0.000000,5.044444
25%,47.000000,5802.799427,44.047619,582.854584,31.000000,9.615625
50%,200.000000,7861.510135,47.916667,2274.306590,54.000000,11.344398
75%,316.000000,10952.937500,52.659574,3760.554431,71.000000,14.564593
max,919.000000,999999.000000,146.875000,10732.268810,90.000000,999999.000000


In [40]:
ltv['value'] = (ltv['driving_period'] * 100 +
               ltv['price'] + 
               ltv['ride_prime_time'] * 50 + 
               ltv['ride_id'] * 5 + 
               2_000 / ltv['drop_requested_lag'] + 
               2_000 / (ltv['delta_requested_at']))
                
ltv.sort_values('value', ascending=False).head()

,driver_id,ride_id,drop_requested_lag,ride_prime_time,price,driving_period,delta_requested_at,value
354,5ccc0e6dc9c7475caf785cdce7b8eb7a,919,5514.246981,47.844828,10732.268810,82.0,13.036997,26073.282452
308,4eb382d1f7d50fae1294964263d1ce82,831,5625.092796,54.609375,9275.832250,81.0,13.292419,24412.118256
401,689bdf87fb2de49f98bf4946cfaa5068,794,6834.735705,50.101215,8595.649229,84.0,10.356423,23664.119448
509,844e9be5a30d8d9c1f8e9ddb086ff717,821,5223.757353,49.460432,9357.989628,72.0,9.077954,23356.708045
722,c07499b5a6f1090f2fb263ec6ac0660c,622,6586.415033,47.916667,8955.817872,86.0,16.697749,23181.731483
